In [1]:
import sys
sys.path.append('../rai/rai/ry')
import numpy as np
import libry as ry

In [2]:
K = ry.Config()
K.addFile("../rai-robotModels/pr2/pr2.g")
K.addFile("../models/tables.g")

K.addFrame("obj0", "table1", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(0 0 .15)>" )
K.addFrame("obj1", "table1", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(0 .2 .15)>" )
K.addFrame("obj2", "table1", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(0 .4 .15)>" )
K.addFrame("obj3", "table1", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(0 .6.15)>" )
K.addFrame("tray", "table2", "type:ssBox size:[.15 .15 .04 .02] color:[0. 1. 0.], logical={ table }, Q:<t(0 0 .07)>" );
K.addFrame("", "tray", "type:ssBox size:[.27 .27 .04 .02] color:[0. 1. 0.]" )
V = K.view()

In [3]:
lgp = K.lgp("../models/fol-pickAndPlace.g")
lgp.nodeInfo()
# this writes the initial state, which is important to check:
# do the grippers have the gripper predicate, do all objects have the object predicate, and tables the table predicate? These need to be set using a 'logical' attribute in the g-file
# the on predicate should automatically be generated based on the configuration

{'boundsConstraints': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsCost': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsFeasible': [1, 1, 1, 1, 1, 1],
 'decision': '<ROOT>',
 'path': '',
 'state': 'STATE_0(START_STATE):{(gripper pr2R):True, (initial pr2R):True, (gripper pr2L):True, (initial pr2L):True, (initial table1):True, (table table2):True, (initial table2):True, (initial table3):True, (object obj0):True, (initial obj0):True, (object obj1):True, (initial obj1):True, (object obj2):True, (initial obj2):True, (object obj3):True, (initial obj3):True, (table tray):True, (initial tray):True, (partOf table2 tray):True, (on table1 obj0):True, (on table1 obj1):True, (on table1 obj2):True, (on table1 obj3):True}'}

In [4]:
lgp.getDecisions()
# This is also useful to check: inspect all decisions possible in this node, which expands the node.
# If there is no good decisions, the FOL rules are buggy

['(grasp pr2R obj0)',
 '(grasp pr2R obj1)',
 '(grasp pr2R obj2)',
 '(grasp pr2R obj3)',
 '(grasp pr2L obj0)',
 '(grasp pr2L obj1)',
 '(grasp pr2L obj2)',
 '(grasp pr2L obj3)']

In [5]:
lgp.walkToDecision(3)
lgp.nodeInfo()
# Using getDecisions and walkToDecision and walkToParent, you can walk to anywhere in the tree by hand

{'boundsConstraints': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsCost': [0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsFeasible': [1, 1, 1, 1, 1, 1],
 'decision': '(grasp pr2R obj3)',
 'path': '(grasp pr2R obj3) ',
 'state': 'STATE_4(STATE_0):{(gripper pr2R):True, (initial pr2R):True, (gripper pr2L):True, (initial pr2L):True, (initial table1):True, (table table2):True, (initial table2):True, (initial table3):True, (object obj0):True, (initial obj0):True, (object obj1):True, (initial obj1):True, (object obj2):True, (initial obj2):True, (object obj3):True, (initial obj3):True, (table tray):True, (initial tray):True, (partOf table2 tray):True, (on table1 obj0):True, (on table1 obj1):True, (on table1 obj2):True, (on table1 obj3):True, decision(grasp pr2R obj3):True, (grasped pr2R obj3):True, (held obj3):True, (busy pr2R):True, komo(touch pr2R obj3):True, (stable pr2R obj3):True}'}

In [6]:
lgp.viewTree()

In [7]:
lgp.walkToNode("(grasp pr2R obj0) (grasp pr2L obj1) (place pr2R obj0 tray)")
lgp.nodeInfo()

{'boundsConstraints': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsCost': [0.30000000000000004, 0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsFeasible': [1, 1, 1, 1, 1, 1],
 'decision': '(place pr2R obj0 tray)',
 'path': '(grasp pr2R obj0) (grasp pr2L obj1) (place pr2R obj0 tray) ',
 'state': 'STATE_15(STATE_9):{(gripper pr2R):True, (initial pr2R):True, (gripper pr2L):True, (initial pr2L):True, (initial table1):True, (table table2):True, (initial table2):True, (initial table3):True, (object obj0):True, (initial obj0):True, (object obj1):True, (initial obj1):True, (object obj2):True, (initial obj2):True, (object obj3):True, (initial obj3):True, (table tray):True, (initial tray):True, (partOf table2 tray):True, (on table1 obj0):True, (on table1 obj1):True, (on table1 obj2):True, (on table1 obj3):True, (grasped pr2L obj1):True, (held obj1):True, (busy pr2L):True, (stable pr2L obj1):True, decision(place pr2R obj0 tray):True, (placed obj0 tray):True, (on obj0 tray):True, komo(touch pr2R obj0):True, (notheld

In [9]:
# at a node, you can compute bounds, namely BT.seq (just key frames), BT.path (the full path),
# and BT.setPath (also the full path, but seeded with the BT.seq result)
lgp.optBound(ry.BT.seq, True)
lgp.nodeInfo()

{'boundsConstraints': [0.0, 0.0, 0.2916793864729323, 0.0, 0.0, 0.0],
 'boundsCost': [0.30000000000000004, 0.0, 15.196287659379943, 0.0, 0.0, 0.0],
 'boundsFeasible': [1, 1, 1, 1, 1, 1],
 'decision': '(place pr2R obj0 tray)',
 'path': '(grasp pr2R obj0) (grasp pr2L obj1) (place pr2R obj0 tray) ',
 'state': 'STATE_15(STATE_9):{(gripper pr2R):True, (initial pr2R):True, (gripper pr2L):True, (initial pr2L):True, (initial table1):True, (table table2):True, (initial table2):True, (initial table3):True, (object obj0):True, (initial obj0):True, (object obj1):True, (initial obj1):True, (object obj2):True, (initial obj2):True, (object obj3):True, (initial obj3):True, (table tray):True, (initial tray):True, (partOf table2 tray):True, (on table1 obj0):True, (on table1 obj1):True, (on table1 obj2):True, (on table1 obj3):True, (grasped pr2L obj1):True, (held obj1):True, (busy pr2L):True, (stable pr2L obj1):True, decision(place pr2R obj0 tray):True, (placed obj0 tray):True, (on obj0 tray):True, komo(t

In [11]:
komo = lgp.getKOMOforBound(ry.BT.seq)
komo.display()

In [12]:
komo=0

In [13]:
lgp.optBound(ry.BT.path, True)
lgp.nodeInfo()

{'boundsConstraints': [0.0,
  0.0,
  0.2916793864729323,
  0.0772998019221563,
  0.0,
  0.0],
 'boundsCost': [0.30000000000000004,
  0.0,
  15.196287659379943,
  2.585855291035301,
  0.0,
  0.0],
 'boundsFeasible': [1, 1, 1, 1, 1, 1],
 'decision': '(place pr2R obj0 tray)',
 'path': '(grasp pr2R obj0) (grasp pr2L obj1) (place pr2R obj0 tray) ',
 'state': 'STATE_15(STATE_9):{(gripper pr2R):True, (initial pr2R):True, (gripper pr2L):True, (initial pr2L):True, (initial table1):True, (table table2):True, (initial table2):True, (initial table3):True, (object obj0):True, (initial obj0):True, (object obj1):True, (initial obj1):True, (object obj2):True, (initial obj2):True, (object obj3):True, (initial obj3):True, (table tray):True, (initial tray):True, (partOf table2 tray):True, (on table1 obj0):True, (on table1 obj1):True, (on table1 obj2):True, (on table1 obj3):True, (grasped pr2L obj1):True, (held obj1):True, (busy pr2L):True, (stable pr2L obj1):True, decision(place pr2R obj0 tray):True, (pl

In [14]:
lgp.viewTree()

In [15]:
# finally, the full multi-bound tree search (MBTS)
# you typically want to add termination rules, i.e., symbolic goals
print("THIS RUNS A THREAD. CHECK THE CONSOLE FOR OUTPUT. THIS IS GENERATING LOTS OF FILES.")
lgp.addTerminalRule("(on obj0 tray) (on obj1 tray) (on obj2 tray)")
lgp.run(2)

THIS RUNS A THREAD. CHECK THE CONSOLE FOR OUTPUT. THIS IS GENERATING LOTS OF FILES.


In [16]:
# wait until you have some number of solutions found (repeat executing this line...)
lgp.numSolutions()

0

In [18]:
# query the optimization features of the 0. solution
lgp.getReport(0, ry.BT.seqPath)

RuntimeError: ../../rai/Core/array.tpp:elem:687(-2) CHECK failed: 'i>=0 && i<(int)N' range error (0>=0)

In [19]:
# get the KOMO object for the seqPath computation of the 0. solution
komo = lgp.getKOMO(0, ry.BT.seqPath)

RuntimeError: ../../rai/Core/array.tpp:elem:687(-2) CHECK failed: 'i>=0 && i<(int)N' range error (0>=0)

In [21]:
komo.displayTrajectory() #SOOO SLOOOW (TODO: add parameter for display speed)

In [22]:
# assign K to the 20. configuration of the 0. solution, check display
# you can now query anything (joint state, frame state, features)
X = komo.getConfiguration(20)
K.setFrameState(X) 

In [20]:
lgp.stop() #stops the thread... takes a while to finish the current job

In [19]:
lgp.run(2) #will continue where it stopped

In [ ]:
komo=0
lgp=0

In [1]:
import sys
sys.path.append('../rai/rai/ry')
import numpy as np
import libry as ry

C = ry.Config()
D = C.view()

C.addFile('../test/lgp-example.g');

lgp = C.lgp("../test/fol.g");

In [ ]:
lgp.walkToNode("(grasp baxterR stick) (push stickTip redBall table1) (grasp baxterL redBall) ");
print(lgp.nodeInfo())

In [3]:
lgp.optBound(BT.pose, True);

RuntimeError: objective.cpp:setCostSpecs:17(-2) CHECK_GE failed: 'toStep'=1 'fromStep'=2 -- 

In [ ]:
komo = lgp.getKOMOforBound(BT.path)
komo.display()

input("Press Enter to continue...")